In [ ]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sat Oct  9 08:19:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

In [ ]:
!pip install -U keras-cv-attention-models

     |████████████████████████████████| 119 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 44.4 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime


In [ ]:
batch_size = 512
imgSize = 224

# Construct a tf.data.Dataset
(train_ds, valid_ds) , ds_info = tfds.load('plant_village', split=['train[:80%]', 'train[80%:]'], shuffle_files=True, as_supervised=True, with_info=True)

NUM_CLASSES = ds_info.features["label"].num_classes
classArr = ds_info.features["label"].names

print(NUM_CLASSES)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/plant_village/1.0.2.incompleteY38IG2/plant_village-train.tfrecord


  0%|          | 0/54303 [00:00<?, ? examples/s]

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.
38


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
  ]
)

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(imgSize, imgSize),
  #  layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)
  layers.experimental.preprocessing.Rescaling(1./255)
])

expendRound = 0
temp_ds = train_ds
for i in range(expendRound):
    train_ds = train_ds.concatenate(temp_ds)

train_ds = train_ds.map(lambda image,label:(resize_and_rescale(image),label)).batch(64)
train_ds = train_ds.map(lambda image,label:(data_augmentation(image),label))


valid_ds = valid_ds.map(lambda image,label:(resize_and_rescale(image),label)).batch(64)

In [ ]:
#https://github.com/leondgarse/keras_cv_attention_models#coatnet
#from keras_cv_attention_models import resnest
#baseModel = resnest.ResNest50(num_classes=NUM_CLASSES)

from keras_cv_attention_models import volo
baseModel = volo.VOLO_d1(pretrained="imagenet", num_classes=NUM_CLASSES)

baseModel.trainable = True
print("Layers count", len(baseModel.layers))

fineTunePresentages = 0.2
fine_tune_at = int( len(baseModel.layers) * fineTunePresentages) # 20
for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable = False

epochsRound = 5
base_learning_rate = 0.0001

baseModel.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = baseModel.fit(train_ds, epochs=epochsRound, validation_data=valid_ds )

test_loss, test_acc = baseModel.evaluate(valid_ds, verbose=2)
print(test_acc)


now = datetime.now()
current_time = now.strftime("%b-%d-%Y_%H:%M:%S")
srt = "/content/drive/MyDrive/savedModel/" + "efficientnetV1B0" + current_time + ".h5"

baseModel.save(srt)
baseModel.save('/content/drive/MyDrive/savedModel/attend.h5')


107274240/107263744 [==============================] - 2s 0us/step
>>>> Load pretrained from: /root/.keras/models/volo_d1_224.h5


Layers count 238


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5
679/679 [==============================] - 1168s 2s/step - loss: 0.2181 - accuracy: 0.9389 - val_loss: 0.0788 - val_accuracy: 0.9732
Epoch 2/5
679/679 [==============================] - 1133s 2s/step - loss: 0.0386 - accuracy: 0.9872 - val_loss: 0.0430 - val_accuracy: 0.9865
Epoch 3/5
679/679 [==============================] - 1131s 2s/step - loss: 0.0267 - accuracy: 0.9916 - val_loss: 0.0519 - val_accuracy: 0.9828
Epoch 4/5
679/679 [==============================] - 1130s 2s/step - loss: 0.0236 - accuracy: 0.9927 - val_loss: 0.0364 - val_accuracy: 0.9892
Epoch 5/5
679/679 [==============================] - 1131s 2s/step - loss: 0.0223 - accuracy: 0.9926 - val_loss: 0.0659 - val_accuracy: 0.9809
170/170 - 67s - loss: 0.0659 - accuracy: 0.9809
0.9809409976005554


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
